## Import libraries

In [14]:
from pathlib import Path
import pandas as pd
import math
import openpyxl

# Create peak pressure file - Attempt 1 (without peak index)

## Access input folders

In [19]:
# Get the current working directory
current_dir = Path.cwd()

# Access the desired directory relative to the current working directory
input_dir1 = current_dir.parent.parent / 'LSDYNA3D' / 'RP_results_all' / 'Tracers_extra'/'TNT'

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

2 [WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/01.5m01.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/01.5m05.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/01.5m09.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/01.5m13.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/01.5m17.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/01.5m21.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/02.5m01.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/02.5m05.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/02.5m09.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/02.5m13.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/02.5m17.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_results_all/Tracers_extra/TNT/02.5m21.5kg'), WindowsPath('G:/Chamodi/L

## Dataframe creation

In [20]:
# Define the empty dataframe
df = pd.DataFrame(columns = ['Explosive type',
                             'Explosive mass',
                             'Perpendicular standoff distance',
                             'Incident angle',
                             'Peak reflected pressure'])

# Access each folder inside input_dir2
for folder in input_dir2:
    #print ("folder", folder)
    
    # obtain explosive mass and perpendicular standoff distance
    p_sd = float(folder.name.split("m")[0])
    e_mass = float(folder.name.split("m")[1].split("kg")[0])
    #print (p_sd,e_mass)
    
    # Make a list of data file names, ignoring files named 'trhist'
    files = [file for file in folder.rglob("*") if not file.name == 'trhist']
        
    # Access each file in the list 
    for file in files:
        #print ("file", file)
        
        #read the incident angle
        if file.stem == '1':
            i_angle = 0
        elif file.stem == '2':
            i_angle = 15
        elif file.stem == '3':
            i_angle = 30
        elif file.stem == '4':
            i_angle = 45
        elif file.stem == '5':
            i_angle = 60
        else:
            continue
        
        # Open the text file
        with open(file, 'r') as filetxt:
            
            # Read the 4th row (index 3, 0-based indexing)
            row4 = filetxt.readlines()[3]
            filetxt.seek(0)
            #print (row4)

            # Check if "Pressure" is in the 4th row
            if "Pressure" in row4:
                # Read the 8th row (index 7, 0-based indexing)
                row8 = filetxt.readlines()[7]
                #print (row8)
                
                # Split the row into parts based on whitespace
                parts = row8.split()
                
                # read peak reflected pressure
                pr_pressure = float(parts[2])
                #print (pr_pressure)
                
                # append rows to an empty DataFrame
                new_row = {'Explosive type': ['TNT'],
                           'Explosive mass' :[e_mass],
                           'Perpendicular standoff distance' : [p_sd],
                           'Incident angle' : i_angle,
                           'Peak reflected pressure' : [pr_pressure/1000]}
                new_row_data = pd.DataFrame(new_row)
                df = pd.concat([df, new_row_data], ignore_index = True)
            else:
                error_message = f'Error: {row4}, {p_sd}, {e_mass}'
                raise ValueError(error_message)
           
print (df)     

    Explosive type  Explosive mass  Perpendicular standoff distance  \
0              TNT             1.5                              1.5   
1              TNT             1.5                              1.5   
2              TNT             1.5                              1.5   
3              TNT             1.5                              1.5   
4              TNT             1.5                              1.5   
..             ...             ...                              ...   
325            TNT            21.5                             20.0   
326            TNT            21.5                             20.0   
327            TNT            21.5                             20.0   
328            TNT            21.5                             20.0   
329            TNT            21.5                             20.0   

    Incident angle  Peak reflected pressure  
0                0                 2398.400  
1               15                 2457.580  
2        

## Create peak pressure excel file

In [21]:
with pd.ExcelWriter('Peak_Reflected_Pressure_TNT_extra.xlsx', engine = "openpyxl", mode='a', if_sheet_exists='replace') as writer:  
    df.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)